In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np

# Define transformations for the training and testing data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load the training and test datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

# Load the ResNet18 model
model = resnet18(pretrained=False, num_classes=10)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training the model
for epoch in range(10):  # Number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'[Epoch: {epoch + 1}, Mini-batch: {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Define a hook function to capture the output of each layer
activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# Register hooks for each layer in the ResNet18 model
for name, layer in model.named_children():
    if isinstance(layer, nn.Sequential):
        for sub_name, sub_layer in layer.named_children():
            sub_layer.register_forward_hook(get_activation(f'{name}_{sub_name}'))
    else:
        layer.register_forward_hook(get_activation(name))

# Get a random image from the test set
dataiter = iter(testloader)
images, labels = dataiter.next()
image = images[0].unsqueeze(0).to(device)

# Pass the image through the model in evaluation mode
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    output = model(image)

# Print the outputs of each layer
for layer_name, layer_output in activation.items():
    print(f'{layer_name}: {layer_output.shape}')

    # Convert the output tensor to a NumPy array
    output_np = layer_output.cpu().numpy()

    # Print the size and values of the output tensor
    print(f'{layer_name} - size: {layer_output.size()}')
    if len(output_np.shape) == 4:  # For 4D tensors
        batch_size, num_channels, height, width = output_np.shape
        for b in range(batch_size):
            for c in range(num_channels):
                print(f'Batch {b}, Channel {c}, Values:\n', output_np[b, c, :, :])
    elif len(output_np.shape) == 2:  # For 2D tensors
        print('Values:\n', output_np)
